In [1]:
ZK_HOST = "localhost:9990"

index = input("Index: ") or "core17"
field = input("Field: ") or "contents"
query = input("Query: ") or "Obama"

df = spark.read.format("solr") \
    .option("zkhost", ZK_HOST) \
    .option("collection", index) \
    .option("query", "{}:{}".format(field, query)) \
    .option("max_rows", "1000") \
    .load()

df.printSchema()

Index: 
Field: 
Query: 
root
 |-- id: string (nullable = false)
 |-- raw: string (nullable = true)



In [2]:
import time

from allennlp.predictors.predictor import Predictor
from spacy import load

acc = sc.accumulator(0)

start = time.time()

def run(part):
    predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/openie-model.2018-08-20.tar.gz")
    nlp = load("en", disable=["ner", "tagger"])
    for d in part:
        doc = nlp(d.raw)
        words = 0
        for sent in doc.sents:
            words += len(sent)
            print(predictor.predict(sentence=sent.text))
        acc.add(words)
    
df.limit(100).rdd.repartition(1).foreachPartition(run)

duration = time.time() - start

print("Took {}s @ {} token/s".format(duration, (acc.value / duration)))

Took 733.0682165622711s @ 145.9373051278814 token/s
